# 量化原理
量化通常是指从浮点数到整数的过程，比较常用的例如8bit量化，即**INT8**量化。在神经网络模型中，通常需要对两种类型的数据进行量化，分别是**权重量化**和**激活值量化**。
激活和权重如下图所示：
<p align="center">
    <img src="./_img/weight_activations_map.png" width="49%"/>
</p>

## 模型的常用表示精度
1. 单精度（常规精度）：用`FP32`表示。
2. 低精度：通常指`INT8`，有时也`FP16`
3. 混合精度：同时使用`FP32`、`FP16`或`BF16`。

## 模型量化的优点
1. 减少存储空间、减少内存占用、减少设备功耗。
2. 加快推理速度。
3. 可以适配多种资源有限的设备，例如手机等。

## 数的表示
需要的基础[IEEE-754](https://en.wikipedia.org/wiki/IEEE_754)基础，关于`FP32`和`FP16`的表示，直接看下面这两个例子：
<p align="center">
    <img src="./_img/FP16_exam.png" width="49%"/>
    <img src="./_img/FP32_exam.png" width="49%"/>
</p>

其余的表示方式如下图所示，重点需要关注`BF16`，需要注意他的指数位与`FP32`相同，位数相同，但是比`FP16`能表示的范围广非常多。目前的GPU也都支持`BF16`。
<p align="center">
    <img src="./_img/2.png"  width="49%"/>
</p>

## 计算模型推理所占用的内存
通用公式为：
$$ memory = \frac{nr_{bits}}{8} \times params $$
`8bit`占一个字节，如果一个模型为70B大小，那么用`FP32`表示就需要280GB内存，用`FP16`表示就要140GB内存。

**但是，精度的降低，往往会导致模型的准确性降低，所以，量化的真正目标在于如何在保持准确性的前提下，尽可能的降低模型精度。**


## 量化的类型
首先可以分为**对称量化**、**非对称量化**和**随机量化**。
### 1. 对称量化
原本的浮点数被量化之后的值是以零为原点中心对称的，量化前后的数值空间都以零点为中心对称。所以量化之后的零点对应量化之前原始值的零点，也就是量化操作并不会改变数据的零点。如下图所示：
<p align="center">
    <img src="./_img/q1.png" width="49%"/>
</p>

量化公式为：
$$ Q_{int} = round[\frac{float}{scale}]$$
$$ scale = \frac{2 \cdot max(|r_{nub}|, r_{max}) }{Q_{max}-Q_{min}} $$
以下图为例，在原本的浮点数列表中，选取绝对值最大的数作为量化前的数值范围，也就是下图中绝对值最大为`10.8`，那么量化前的浮点数范围为`[-10.8,10.8]`。注意：**哪怕复数中最小值为-0.001，浮点数空间也不会因此改变，这就导致了可能会有部分空间是浪费的。**
<p align="center">
    <img src="./_img/Symmetric_Quantization_exam.png" width="49%"/>
</p>

反量化的公式如下：
$$ Q_{int} = float \times scale$$
反量化并不能完美还原到原数值，往往会存在误差，这就是量化带来的精度损失。如下图所示：
<p align="center">
    <img src="./_img/量化反量化精度损失示例图.png" width="49%"/>
</p>
这被称为**量化误差**.

In [ ]:
import numpy as np

def saturete(x):
    return np.clip(x, -127, 127)

def scale_cal(x):
    max_val = np.max(np.abs(x))
    return max_val / 127

def quant_float_data(x, scale):
    xq = saturete(np.round(x/scale))
    return xq

def dequant_data(xq, scale):
    x = (xq * scale).astype('float32')
    return x

if __name__ == "__main__":
    np.random.seed(1)
    data_float32 = np.random.randn(3).astype('float32')
    print(f"input = {data_float32}")

    scale = scale_cal(data_float32)
    print(f"scale = {scale}")

    data_int8 = quant_float_data(data_float32, scale)
    print(f"quant_result = {data_int8}")
    data_dequant_float = dequant_data(data_int8, scale)
    print(f"dequant_result = {data_dequant_float}")

    print(f"diff = {data_dequant_float - data_float32}")


### 2. 非对称量化
非对称量化则不是围绕零点对称，而是选取其中的`[最小值,最大值]`作为浮点数范围进行量化，量化之后，原浮点数中的零点并不一定对应量化之后的零点。如下图所示：
<p align="center">
    <img src="./_img/q2.png" width="49%"/>
</p>
量化公式为：
首先需要计算出**缩放因子**和**零点**。

$$ S = \frac{ R_{max}-R_{min} }{ Q_{max}-Q_{min} } $$
$$ Z = Q_{max} - \frac{ R_{max} }{S} $$

其中，$R_{max}$ 表示输入浮点数据中的最大值，$R_{min}$，表示输入浮点数据中的最小值，$Q_{max}$表示最大的定点值（127 / 255），$Q_{min}$表示最小的定点值（-128 / 0）。
然后计算量化后的值：

$$ Q_{quantized} = round( \frac{float}{scale} ) + offset $$

也可以直接看下图：
<p align="center">
    <img src="./_img/q22.png" width="49%"/>
</p>

反量化公式为：
<p align="center">
    <img src="./_img/dq1.png" width="49%"/>
</p>

In [ ]:
import numpy as np

def saturete(x, int_max, int_min):
    return np.clip(x, int_min, int_max)

def scale_z_cal(x, int_max, int_min):
    scale = (x.max() - x.min()) / (int_max - int_min)
    z = int_max - np.round((x.max() / scale))
    return scale, z

def quant_float_data(x, scale, z, int_max, int_min):
    xq = saturete(np.round(x/scale + z), int_max, int_min)
    return xq

def dequant_data(xq, scale, z):
    x = ((xq - z)*scale).astype('float32')
    return x

if __name__ == "__main__":
    np.random.seed(1)
    data_float32 = np.random.randn(3).astype('float32')
    int_max = 127
    int_min = -128
    print(f"input = {data_float32}")

    scale, z = scale_z_cal(data_float32, int_max, int_min)
    print(f"scale = {scale}")
    print(f"z = {z}")
    data_int8 = quant_float_data(data_float32, scale, z, int_max, int_min)
    print(f"quant_result = {data_int8}")
    data_dequant_float = dequant_data(data_int8, scale, z)
    print(f"dequant_result = {data_dequant_float}")
    
    print(f"diff = {data_dequant_float - data_float32}")


## 一些基本概念
### 离群值
接下来要考虑一个非常重要的影响因素，加入一个数组中有一个非常大的值为100，而其他的值都是1左右，那么这个数组直接按照前面的量化方式就会导致所有在1左右的值都被映射到同一个值，如下图所示：

<img src="./_img/list_out.png" width="60%"/>
<img src="./_img/quat_list_out.png" width="60%"/>

这就是离群值，也称为异常值，如果不进行特殊处理，会导致其他数值失效。

一种处理方法就是**截断**，设定一个浮点数范围，将超出这个范围的数直接映射到最大值。如下图所示，手动将范围设置为`[-5,5]`，超出这个范围的数就直接量化为了127。

<img src="./_img/truncate_quan_exam.png" width="60%"/>

能够显著降低非离群值的误差，但是离群值的误差增加。

### 校准
其中，找到这个动态范围的过程叫做**校准**，**目的就是找到一个包含尽可能多值的范围，同时最小化量化误差。**。

LLM的参数主要由`weight`和`bias`组成，而`bias`的数量级远远小于`weight`，所以`bias`通常使用精度较高的量化（如`INT16`）存储，`weights`也就成了重点量化对象。

在对`weights`进行量化时，可以选择的校准技术包括：
1. 手动输入动态范围的百分位
2. 优化原始`weights`和量化`weights`之间的均方误差。(MSE)
3. 基于KL散度
   


### 激活
前面所说的`weights`和`bias`都是在模型训练完成之后就保持不变的静态值，而在模型部署推理的计算过程中，还有会持续更新的输入`input`，这些值被称为**激活值**，因为它们往往会经过一些**激活函数**。激活值是动态变化的，所以量化起来会更加困难。

如下图所示，这些激活值是在每个隐藏层之后更新的，只有在模型计算完成时才知道具体的值。
<img src="./_img/activation_value.png" width="60%"/>

----
下面则进入到具体的量化方法。
主要分为三种：
|量化方法| 特点 | 适用场景 | 使用条件 | 易用性 | 精度损失 | 预期收益 |
| -- | -- | -- | -- | -- | -- | -- |
| 量化训练 | 通过在SFT微调过程中通过Fake Quant来降低量化带来的误差 | 对量化敏感的场景、模型，例如目标检测、分割、OCR 等 | 有大量带标签数据 | 好 | 极小 | 减少存续空间 4X，降低计算内存 |
| 静态离线量化 (PTQ Static) | 通过少量校准数据得到量化模型，预先计算好每个隐藏层中激活值的缩放因子和零点，不重复计算 | 对量化不敏感的场景，例如图像分类任务 | 有少量无标签数据 | 较好 | 较少 | 减少存续空间 4X，降低计算内存 |
| 动态离线量化 (PTQ Dynamic) | 对激活值的量化是在每个隐藏层之后都会重新计算 | 模型体积大、访存开销大的模型，例如 BERT 模型 | 无 | 一般 | 一般 | 减少存续空间 2/4X，降低计算内存 |

# Post-Training Quantization 训练后量化
**PTQ**是在目前最受欢迎的量化技术，模型在训练完成之后，直接对`weights`、`bias`以及`activations`进行量化。其中，对`weights`的量化主要是通过对称或非对称量化，因为`weights`在推理过程中是保持不变的，量化起来容易。

但是，由于激活值需要在每个隐藏层计算完成之后，再通过激活函数得到，在推理过程中是动态变化，无法事先知道他们的分布，所以，对于激活值的量化就分为了两种形式：
- 动态量化（Dynamic Quantization）
- 静态量化（Static Quantization）
  
所以，动态和静态的量化方式主要区别就是对激活值的量化方式。


## 动态量化
如下图所示，在每一个隐藏层之后，计算得到激活值时，才收集这一层的激活值进行量化。使用该层的激活值分布来计算量化所述的零点和缩放因子。
<p align="center">
    <img src="./_img/dyna_quan.png" width="60%"/>
    <img src="./_img/dyna_quan2.png" width="60%"/>
</p>

这个量化过程是在前向传播时，数据每通过一个隐藏层都会重复，每层都会单独计算零点和缩放因子。所以，**动态是指在前向传播计算过程中，动态的获取每一层激活值的零点和激活因子来进行量化**。

## 静态量化
知道了动态量化之后，静态量化就是想办法提前计算好每一层激活值的零点和缩放因子，而在真正进行推理部署的时候，不再去重复计算。所以，为了能够提前计算这些值，就需要使用一个**校准数据集**，来计算激活值的潜在分布。
<p align="center">
    <img src="./_img/static_quant.png" width="60%"/>
</p>

其实，也就是使用一个数据集来进行前向推理，收集每层的激活值，然后计算出能使每条样本数据的最终误差都较小的零点和缩放因子。而在实际推理过程中，便可以不用重复计算。

显然，静态方法的准确性会低于动态方法，因为静态方法是拿少部分数据计算得到的零点和缩放因子来代替全部数据。但是静态量化可以缩短在推理时的时间和计算量。

# Quantization Aware Training 量化感知训练
训练后量化都是在模型训练完成之后，并没有让模型在训练过程中学习到量化带来的误差，这也就是QAT着力点。

QAT的工作原理如下图所示，在训练过程中，在计算图中增加一个`Fake quant`，也称为**伪量化节点**，它对参数进行量化并立即反量化，如下图中，将`FP32`量化成`INT4`，然后又立即反量化为`FP32`。这么做，是因为反量化之后得到的数据与原数据相比可能是有误差的，就是将这个误差作为噪声，带入到训练过程中。最近在计算损失loss时，也就是考虑到了量化带来的误差，使得模型在后续的量化过程中，不会很明显的降低模型精度。
<p align="center">
    <img src="./_img/QAT.png" width="60%"/>
</p>

`FakeQuant`节点通常在以下3个位置插入：
1. **卷积层（Conv2D）前后**：这可以帮助卷积操作在量化后适应低精度计算。
2. **全连接层（Fully Connected Layer）前后**：这对于处理密集矩阵运算的量化误差非常重要。
3. **激活函数（如 ReLU）前后**：这有助于在非线性变换中保持量化精度。

下面就是对输入和`Weights`都插入了`FakeQuant`伪量化算子。

#### 伪量化节点的作用：
1. 找到输入数据的分布，即找到 MIN 和 MAX 值；
2. 模拟量化到低比特操作的时候的精度损失，把该损失作用到网络模型中，传递给损失函数，让优化器去在训练过程中对该损失值进行优化。

## 划重点：为什么QAT损失更低？
下面的才是重点，为什么QAT能够有效的降低在低精度下的损失误差？
首先来看这个`loss-weight`关系图，
<p align="center">
    <img src="./_img/loss_weight.png" width="60%"/>
</p>

在不考虑量化的情况下，根据梯度下降算法应该选择`loss`最小的权重，就是图中的`B`点，而这个点位于`loss`梯度更大的低谷中，这也被称为`narrow minima`，`B`点量化成`Int4`之后的值可能是图中的`b`点，可以看到loss的变化是非常大的，就导致误差较大。

就如下图所示，在考虑量化之后的极值点选择，在训练过程中考虑量化，则模型会选择量化后损失更小的`a`点，而不是`b`点。这就是为什么`QAT`能够比`PTQ`带来更低的精度损失。

<p align="center">
    <img src="./_img/loss_weight2.png" width="60%"/>
</p>

这一点，我认为至关重要，是理解`QAT`的关键。


## 前向传播
需要注意：**我们在计算图中加入一个节点，就必须考虑前向传播和反向传播两个过程的计算。

前向传播过程中，`FakeQuant`就是完成量化和反量化操作，引入量化误差。输入的是`tensor`，量化时需要计算出`Min`和`Max`值，然后计算出缩放因子`S`和零点`Z`。

## 反向传播
在前向传播过程中，模型将量化带来的误差向前传递，然后再将梯度反向传递用来更新模型参数，模型参数的更新因此包含了量化带来的误差。但是因为量化后的`Weights`是离散的，对`W`求导为0：
$$ \frac{ \partial Q(W)}{ \partial  W} = 0$$

因为梯度为0，导致模型`Weights`不会更新。所以，在反向传播的过程中，可以使用 **直通估计器（Straight-Through Estimator，简称 STE）** 来简单的将梯度通过量化传递，近似来计算梯度。STE近似假设量化操作的梯度为1，从而允许梯度直接通过量化节点，如下图所示：
<p align="center">
    <img src="./_img/quan_back_ward.png" width="60%"/>
</p>

### 量化感知训练的技巧
1. 从已校准的表现最佳的 PTQ 模型开始
    与其从未训练或随机初始化的模型开始感知量化训练，不如从已校准的 PTQ 模型开始，这样能为 QAT 提供更好的起点。特别是在低比特宽量化情况下，从头开始训练可能会非常困难，而使用表现良好的 PTQ 模型可以帮助确保更快的收敛和更好的整体性能。
2. 微调时间为原始训练计划的 10%
    感知量化训练不需要像原始训练那样耗时，因为模型已经相对较好地训练过，只需要调整到较低的精度。一般来说，微调时间为原始训练计划的 10% 是一个不错的经验法则。
3. 使用从初始训练学习率 1% 开始的余弦退火学习率计划
4. 为了让 STE 近似效果更好，最好使用小学习率。大学习率更有可能增加 STE 近似引入的方差，从而破坏已训练的网络。
5. 使用余弦退火学习率计划可以帮助改善收敛，确保模型在微调过程中继续学习。从较低的学习率(如初始训练学习率的 1%)开始有助于模型更平稳地适应较低的精度，从而提高稳定性。直到达到初始微调学习率的 1%(相当于初始训练学习率的 0.01%)。在 QAT 的早期阶段使用学习率预热和余弦退火可以进一步提高训练的稳定性。
6. 使用带动量的 SGD 优化器而不是 ADAM 或 RMSProp
    尽管 ADAM 和 RMSProp 是深度学习中常用的优化算法，但它们可能不太适合量化感知微调。这些方法会按参数重新缩放梯度，可能会扰乱感知量化训练的敏感性。使用带动量的 SGD 优化器可以确保微调过程更加稳定，使模型能够更有控制地适应较低的精度。

通过 QAT，神经网络模型能够在保持高效推理的同时，尽量减少量化带来的精度损失，是模型压缩和部署的重要技术之一。在大多数情况下，一旦应用感知量化训练，量化推理精度几乎与浮点精度完全相同。然而，在 QAT 中重新训练模型的计算成本可能是数百个 epoch。